In [19]:
#Autor: Andrés
#Tarea: https://github.com/UCM-GIDIA-PD1/c2425-R4/issues/15
#Objetivo: crear particiones de datos para entrener modelos (train, validation, test)
#Descripción y uso: en primer lugar lee el data frame de peleas_ponderadas de la carpeta data se dividen el conjuntos
#de datos en tres subconjuntos y se guardan en parquets. Para usarlo simplemente ejecutar el archivo.
#Estado actual: terminado

## Partición datos **Ponderados**

* [Carga de los datos](#carga-de-los-datos)  
* [Partición de los datos](#particion-de-los-datos)
* [Guardar datos](#guardar-datos)

In [20]:
import pandas as pd
import os

### Carga de los datos

In [21]:
ruta = os.path.join(os.getcwd(), "..", "..", "data", "processed", "peleas_ponderadas.parquet")
df = pd.read_parquet(ruta)

In [22]:
df.head()

,DATE,Peleador_A,Peleador_B,WINNER,KD_A,KD_B,SIG_STR_A,SIG_STR_B,TD_PORC_A,TD_PORC_B,...,Derrotas_Sub_A,Derrotas_Sub_B,Derrotas_Decision_A,Derrotas_Decision_B,KD_DIFF,SIG_STR_DIFF,TD_DIFF,SUB_ATT_DIFF,REV_DIFF,CTRL_DIFF
0,2000-06-09,Kevin Randleman,Pedro Rizzo,False,0.0,0.8,0.279,0.315,0.770,0.000,...,0.0,0.0,0.8,0.0,-0.8,-0.036,0.615385,1.0,0.0,526.0
1,2000-09-22,Jeremy Horn,Eugene Jackson,False,0.0,0.5,0.696,0.550,0.084,0.000,...,0.0,0.5,0.0,0.0,-0.5,0.146,0.250000,0.4,0.0,3.3
2,2001-02-23,Tito Ortiz,Evan Tanner,False,0.3,0.0,0.552,0.741,0.874,1.000,...,0.0,0.0,0.0,0.0,0.3,-0.189,0.178571,-0.1,0.0,458.1
3,2001-05-04,Chuck Liddell,Kevin Randleman,False,0.0,0.0,0.508,0.532,0.000,0.758,...,1.0,0.0,0.0,1.0,0.0,-0.024,-0.518519,-1.5,0.2,-455.0
4,2001-06-29,Pat Miletich,Shonie Carter,False,0.0,1.0,0.451,0.521,0.500,0.646,...,0.5,0.0,0.0,0.0,-1.0,-0.070,0.037945,0.8,-1.2,-137.3


### Filtrado del *dataframe*
Eliminamos datos anteriores al 2010 debido a que siempre ganaba el Peleador A y los datos generaban muchos fallos al entrenar el modelo generando grandes desbalances

Se oberva que en casi todos los años antes del 2010, los combates siempre los ganaba el peleador A

In [23]:
df.groupby(df["DATE"].dt.year)["WINNER"].value_counts(normalize=True).unstack()

WINNER,False,True
DATE,,
2000,1.000000,NaN
2001,1.000000,NaN
2002,1.000000,NaN
2003,0.900000,0.100000
2004,1.000000,NaN
2005,1.000000,NaN
2006,1.000000,NaN
2007,0.976744,0.023256
2008,1.000000,NaN


In [24]:
df = df[df["DATE"].dt.year >= 2010]

### Particion de los datos
Se dividen los datos en train y test, respetando el orden temporal

In [25]:
# Ordenar por fecha (si no está ordenado ya)
df = df.sort_values(by="DATE")

# Definir los tamaños de cada subconjunto
train_size = 0.8  # 80%
test_size = 0.2    # 20%

# Calcular índice de corte
n = len(df)
train_end = int(n * train_size)

# Dividir el DataFrame
df_train = df.iloc[:train_end]
df_test = df.iloc[train_end:]

# Obtener las fechas de corte
train_end_date = df_train["DATE"].max()
test_end_date = df_test["DATE"].max()

print(f"Última fecha en train: {train_end_date}")
print(f"Última fecha en test: {test_end_date}")

# Mostrar tamaños
print(f"Train: {len(df_train)}, Test: {len(df_test)}")


Última fecha en train: 2022-11-12 00:00:00
Última fecha en test: 2025-02-15 00:00:00
Train: 2554, Test: 639


Se verifica la distribución de las clases tras la división. Como se observa están muy poco desbalanceadas

In [26]:
# Verificar tamaños y balance
def check_distribution(data, name):
    total = len(data)
    class_dist = data["WINNER"].value_counts(normalize=True).round(2)
    print(f"{name}: {total} muestras ({total/len(df):.0%}) | Distribución: {class_dist.to_dict()}")

check_distribution(df_train, "Entrenamiento")
check_distribution(df_test, "Prueba")

Entrenamiento: 2554 muestras (80%) | Distribución: {False: 0.56, True: 0.44}
Prueba: 639 muestras (20%) | Distribución: {False: 0.55, True: 0.45}


### Guardar datos
Se guardan los *dataframes* en archivos *parquet*

In [27]:
base_dir = os.path.join(os.getcwd(), "..", "..", "data")
ruta_partition = os.path.join(base_dir, "P2")

os.makedirs(ruta_partition, exist_ok=True)

df_train.to_parquet(os.path.join(ruta_partition, "train.parquet"), index = False)
df_test.to_parquet(os.path.join(ruta_partition, "test.parquet"), index = False)